In [1]:
import os, sys, glob, argparse
import pandas as pd
import numpy as np
# from tqdm import tqdm

import cv2, time
# from PIL import Image
# from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchmetrics
import torchvision.models as models
# import torchvision.transforms as transforms
# import torchvision.datasets as datasets
import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

import albumentations as A


# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [2]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3070 Ti Laptop GPU'

In [3]:
# 读取数据集
train_path = glob.glob("./data/sample/*/*")
test_path = glob.glob("./data/test/*")

train_path.sort()
test_path.sort()

# train_df = pd.read_csv("data/train.csv")
# train_df = train_df.sort_values(by="name")
# train_label = train_df["label"].values

train_label = [np.int64(0) for i in range(480)]+[np.int64(1) for i in range(2400)]

# 自定义数据集
# 带有图片缓存的逻辑
DATA_CACHE = {}


class XunFeiDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None

    def __getitem__(self, index):
        if self.img_path[index] in DATA_CACHE:
            img = DATA_CACHE[self.img_path[index]]
        else:
            img = cv2.imread(self.img_path[index])
            DATA_CACHE[self.img_path[index]] = img
        if self.transform is not None:
            img = self.transform(image=img)["image"]
        img = img.transpose([2, 0, 1])
        return img, torch.from_numpy(np.array(self.img_label[index]))

    def __len__(self):
        return len(self.img_path)

dataset = XunFeiDataset(
    train_path,
    train_label,
    A.Compose(
        [
            A.RandomRotate90(),
            A.Resize(256, 256),
            A.RandomCrop(224, 224),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ]
    ),
)
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset=dataset, lengths=[0.95, 0.05], generator=torch.Generator().manual_seed(42)
)

# 训练集
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=True, num_workers=0, pin_memory=False
)

# 验证集
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=False
)

# 测试集
test_loader = torch.utils.data.DataLoader(
    XunFeiDataset(
        test_path,
        [0] * len(test_path),
        A.Compose(
            [
                A.Resize(256, 256),
                A.RandomCrop(224, 224),
                A.HorizontalFlip(p=0.5),
                A.RandomBrightnessContrast(p=0.5),
                A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ]
        ),
    ),
    batch_size=50,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
)

In [4]:
a=cv2.imread(train_path[0])
np.shape(a)

(64, 112, 3)

In [14]:
class XunFeiNet50(nn.Module):
    def __init__(self):
        super(XunFeiNet50, self).__init__()
        model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out
        
class XunFeiNet101(nn.Module):
    def __init__(self):
        super(XunFeiNet101, self).__init__()
        model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out
    
class XunFeiNet152(nn.Module):
    def __init__(self):
        super(XunFeiNet152, self).__init__()
        model = models.resnet152(weights=models.ResNet152_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out
    
class XunFeiNet34(nn.Module):
    def __init__(self):
        super(XunFeiNet34, self).__init__()
        model = models.resnet34()
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(512, 2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out
    
class XunFeiNet18(nn.Module):
    def __init__(self):
        super(XunFeiNet18, self).__init__()
        model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(512, 2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out
    
model34 = XunFeiNet34().to(device)
model50 = XunFeiNet50().to(device)
model101= XunFeiNet101().to(device)
model152= XunFeiNet152().to(device)
# model = model.to(device)
criterion = nn.CrossEntropyLoss().cuda()

In [25]:
# 模型训练
def train(train_loader, model, criterion, optimizer):
    start = time.time()
    start_batch = [start, 0]
    model.train()
    train_loss = 0.0
    preds = torch.tensor([])
    target_all = torch.tensor([])
    for i, (input, target) in enumerate(train_loader):
        input = input.to(device)
        target = target.to(device)
        output = model(input)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            start_batch[(1+(i + 1) // 100) % 2] = time.time()
            print(
                "Train loss",
                loss.item(),
                "t={}s".format(
                    start_batch[(1+(i + 1) // 100) % 2]
                    - start_batch[((i + 1) // 100) % 2]
                ),
            )

        preds = torch.cat((preds, output.cpu().argmax(1)))
        target_all = torch.cat((target_all, target.cpu()))

        train_loss += loss.item()

    val_acc = torchmetrics.functional.classification.multiclass_f1_score(
        preds, target_all, num_classes=2, average="macro"
    )
    print("t={}s".format(time.time() - start))
    print("F1 score", val_acc)
    return train_loss / len(train_loader)


# 模型验证
def validate(val_loader, model, criterion):
    model.eval()
    val_acc = 0.0
    preds = torch.tensor([])
    target_all = torch.tensor([])
    val_loss=0.0
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            input = input.to(device)
            target = target.to(device)
            output = model(input)
            loss = criterion(output, target)
            # val_acc += (output.argmax(1) == target).sum().item()
            preds = torch.cat((preds, output.cpu().argmax(1)))
            target_all = torch.cat((target_all, target.cpu()))

        val_acc = torchmetrics.functional.classification.multiclass_f1_score(
            preds, target_all, num_classes=2, average="macro"
        )
        val_loss += loss.item()
    # return val_acc / len(val_loader.dataset)
    return val_acc,val_loss/len(val_loader)


# 模型预测
def predict(test_loader, model):
    model.eval()

    test_pred = []
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input = input.to(device)
            output = model(input)
            test_pred.append(output.data.cpu().numpy())

    return np.vstack(test_pred)

In [ ]:
# for best model
model=model152
optimizer = torch.optim.AdamW(model.parameters(), 0.000005)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline_152.pth")

In [27]:
# for best model
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.000005)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline_101.pth")

Train loss 0.59358149766922 t=37.89586925506592s
Train loss 0.5492005944252014 t=65.1143262386322s
Train loss 0.5439359545707703 t=89.79940986633301s
Train loss 0.498115599155426 t=114.49705696105957s
Train loss 0.5084092020988464 t=139.29148244857788s
Train loss 0.535024881362915 t=164.13810873031616s
Train loss 0.4228997528553009 t=189.23814034461975s
Train loss 0.39884406328201294 t=215.17427492141724s
t=233.4092059135437s
F1 score tensor(0.4689)
epoch 1 : train_loss: 0.5194392675577209 val_loss: 0.10963937044143676 val_f1: tensor(0.4504)


RuntimeError: Parent directory ./model does not exist.

In [29]:
# for best model
model=model50
optimizer = torch.optim.AdamW(model.parameters(), 0.000005)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline_50.pth")

Train loss 0.5575281381607056 t=4.891497373580933s
Train loss 0.5705251097679138 t=10.128498792648315s
Train loss 0.5479844212532043 t=15.585680484771729s
Train loss 0.4515620172023773 t=21.141336679458618s
Train loss 0.5779539942741394 t=26.68748378753662s
Train loss 0.44088584184646606 t=32.26030945777893s
Train loss 0.4428062438964844 t=37.89376711845398s
Train loss 0.5259270071983337 t=43.387813329696655s
t=46.95921230316162s
F1 score tensor(0.4660)
epoch 1 : train_loss: 0.5123963730279789 val_loss: 0.10738210678100586 val_f1: tensor(0.4891)
Train loss 0.3900531828403473 t=5.018091440200806s
Train loss 0.45199495553970337 t=10.624507427215576s
Train loss 0.4041074514389038 t=16.265429496765137s
Train loss 0.3333797752857208 t=21.955791473388672s
Train loss 0.45651888847351074 t=27.641228199005127s
Train loss 0.5289812088012695 t=33.328871965408325s
Train loss 0.5401110649108887 t=39.11371374130249s
Train loss 0.3294830322265625 t=44.81885123252869s
t=48.449108600616455s
F1 score te

In [ ]:
# for best model
model=model34
optimizer = torch.optim.AdamW(model.parameters(), 0.000005)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline_34.pth")

In [30]:
model=model50
optimizer = torch.optim.AdamW(model.parameters(), 0.000005)
epochs = 10
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )

Train loss 0.3254185616970062 t=5.227536916732788s
Train loss 0.38939276337623596 t=10.747638940811157s
Train loss 0.47319045662879944 t=16.265472888946533s
Train loss 0.3093191683292389 t=21.72244358062744s
Train loss 0.2806541919708252 t=27.229488134384155s
Train loss 0.2751478850841522 t=32.74290728569031s
Train loss 0.3427433967590332 t=38.25426983833313s
Train loss 0.3619307279586792 t=43.7697811126709s
t=47.32741022109985s
F1 score tensor(0.4684)
epoch 1 : train_loss: 0.3934807222942973 val_loss: 0.08376458287239075 val_f1: tensor(0.4891)
Train loss 0.4363410174846649 t=4.991467714309692s
Train loss 0.4505930244922638 t=10.55420207977295s
Train loss 0.3801509737968445 t=16.1241352558136s
Train loss 0.42548298835754395 t=21.758843421936035s
Train loss 0.3100317120552063 t=27.410313606262207s
Train loss 0.3124871551990509 t=33.33676624298096s
Train loss 0.35863491892814636 t=39.17458248138428s
Train loss 0.3848431408405304 t=44.79841089248657s
t=48.39830183982849s
F1 score tensor(0

In [ ]:
# 对测试集多次预测
pred = None
model=model101
model.load_state_dict(torch.load("./model/baseline_101.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model=model152
model.load_state_dict(torch.load("./model/baseline_152.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += 0.9*predict(test_loader, model)
    print(_+1)
model=model50
model.load_state_dict(torch.load("./model/baseline_50.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model=model34
model.load_state_dict(torch.load("./model/baseline_34.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += 0.9*predict(test_loader, model)
    print(_+1)
submit = pd.DataFrame(
    {
        'name': [x.split('/')[-1] for x in test_path],
        'label': pred.argmax(1)
})

# 生成提交结果
submit = submit.sort_values(by='name')
submit.to_csv('submit6.csv', index=None)

In [ ]:
model.train()
train_loss = 0.0
model = model.to(device)
for i, (input, target) in enumerate(train_loader):
    input = input.to(device)
    target = target.to(device)
    output = model(input)
    loss = criterion(output, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
epochs=4
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.0001)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
torch.save(model.state_dict(),"./model/baseline_test.pth")



In [ ]:
epochs=3
model101= XunFeiNet().to(device)
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.0001)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
torch.save(model.state_dict(),"./model/baseline_101.pth")

In [ ]:
pred = None
model=model101
model.load_state_dict(torch.load("./model/baseline_test.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model.load_state_dict(torch.load("./model/baseline_101.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model=model50
model.load_state_dict(torch.load("./model/baseline_50.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
submit = pd.DataFrame(
    {
        'name': [x.split('/')[-1] for x in test_path],
        'label': pred.argmax(1)
})

# 生成提交结果
submit = submit.sort_values(by='name')
submit.to_csv('submit5.csv', index=None)

In [ ]:
model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
model.fc

In [ ]:
val_acc = validate(val_loader, model)
val_acc

In [ ]:
dataset=XunFeiDataset(train_path[:-1000], train_label[:-1000],
            A.Compose([
            # A.RandomRotate90(),
            A.Resize(256, 256),
            A.RandomCrop(224, 224),
            # A.HorizontalFlip(p=0.5),
            # A.RandomBrightnessContrast(p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        )
train_dataset, test_dataset=torch.utils.data.random_split(dataset=dataset,lengths=[0.9,0.1],generator=torch.Generator().manual_seed(42))
train_dataset

In [ ]:
model.load_state_dict(torch.load("./model/baseline_test.pth"))
val_acc,val_loss = validate(val_loader, model, criterion)
print(val_loss)
len(val_loader.dataset)